# Purpose


This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.



## Imports

In [ ]:
from pprint import pprint
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import ta_lib.eda.api as eda

In [ ]:
import warnings

warnings.filterwarnings('ignore', message="The default value of regex will change from True to False in a future version.", 
                        category=FutureWarning)

In [ ]:
initialize_environment(debug=False, hide_warnings=True)

## Utility functions

# 1. Initialization

In [ ]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
pprint(list_datasets(context))

housing_df = load_dataset(context, 'raw/housing')


# 2. Data cleaning and consolidation

**<u>NOTES</u>**

The focus here is to create a cleaned dataset that is appropriate for solving the DS problem at hand from the raw data.

**1. Do**
* clean dataframe column names
* ensure dtypes are set properly
* join with other tables etc to create features
* transform, if appropriate, datetime like columns to generate additional features (weekday etc)
* transform, if appropriate, string columns to generate additional features
* discard cols that are not useful for training the model (IDs, constant cols, duplicate cols etc)
* additional features generated from existing columns


**2. Don't**
* handle missing values or outliers here. mark them and leave them for processing downstream.


## 2.1 Clean individual tables 

### Products Table

From data discovery, we know the following

* all columns are strings : nothing to fix. Apply generic cleaning (strip extra whitespace etc)
* ensure all `invalid` string entries are mapped to np.NaN
* some column are duplicates (eg. color, Ext_Color). Better to `coalesce` them instead of an outright discard of one of the columns.
* SKU is key column : ensure no duplicate values
* This will go into production code


In [ ]:
housing_df_clean = (
    housing_df

    .copy()

    # set dtypes : nothing to do here
    .passthrough()
    
    .replace({'': np.NaN})
    
    .clean_names(case_type='snake')

)
housing_df_clean.dropna(subset=['total_bedrooms'], inplace=True)

housing_df_clean.reset_index(drop=True,inplace = True)
housing_df_clean.head()

In [ ]:
housing_df_clean.isna().sum()

### NOTE

It's always a good idea to save cleaned tabular data using a storage format that supports the following 

1. preserves the type information
2. language agnostic storage format
3. Supports compression
4. Supports customizing storage to optimize different data access patterns

For larger datasets, the last two points become crucial.

`Parquet` is one such file format that is very popular for storing tabular data. It has some nice properties:
- Similar to pickles & RDS datasets, but compatible with all languages
- Preserves the datatypes
- Compresses the data and reduces the filesize
- Good library support in Python and other languages
- As a columnar storage we can efficiently read fewer columns
- It also supports chunking data by groups of columns (for instance, by dates or a particular value of a key column) that makes loading subsets of the data fast.

In [ ]:
save_dataset(context, housing_df_clean, 'cleaned/housing')

# 3. Generate Train, Validation and Test datasets



- We split the data into train, test (optionally, also a validation dataset)
- In this example, we are binning the target into 10 quantiles and then use a Stratified Shuffle to split the data.
- See sklearn documentation on the various available splitters
- https://scikit-learn.org/stable/modules/classes.html#splitter-classes
- This will go into production code (training only)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from ta_lib.core.api import custom_train_test_split  # helper function to customize splitting
from scripts import *


housing_df_clean["income_cat"] = pd.cut(
    housing_df_clean["median_income"],
    bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
    labels=[1, 2, 3, 4, 5],
)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing_df_clean, housing_df_clean["income_cat"]):
    strat_train_set = housing_df_clean.loc[train_index]
    strat_test_set = housing_df_clean.loc[test_index]


# splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=context.random_seed)
# strat_train_set, strat_test_set = custom_train_test_split(housing_df_clean, splitter, by=binned_selling_price)

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
housing_tr_c = strat_train_set.copy()
housing_tr_c.plot(kind="scatter", x="longitude", y="latitude")


In [ ]:
housing_tr_c.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)


In [ ]:
housing_tr_c.drop("ocean_proximity", axis=1, inplace=True)
corr_matrix = housing_tr_c.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing_tr_features = strat_train_set.drop(
    "median_house_value", axis=1
)  # drop labels for training set
housing_tr_target = strat_train_set["median_house_value"].copy()

In [ ]:
imputer = SimpleImputer(strategy="median")

housing_tr_f_num = housing_tr_features.drop("ocean_proximity", axis=1)

imputer.fit(housing_tr_f_num)
X = imputer.transform(housing_tr_f_num)

housing_tr_fe = pd.DataFrame(X, columns=housing_tr_f_num.columns, index=housing_tr_features.index)
housing_tr_fe["rooms_per_household"] = housing_tr_fe["total_rooms"] / housing_tr_fe["households"]
housing_tr_fe["bedrooms_per_room"] = (
    housing_tr_fe["total_bedrooms"] / housing_tr_fe["total_rooms"]
)
housing_tr_fe["population_per_household"] = (
    housing_tr_fe["population"] / housing_tr_fe["households"]
)

housing_tr_cat = housing_tr_features[["ocean_proximity"]]
housing_tr_features = housing_tr_fe.join(pd.get_dummies(housing_tr_cat, drop_first=True))

In [ ]:
housing_te_features = strat_test_set.drop("median_house_value", axis=1)
housing_te_target = strat_test_set["median_house_value"].copy()

housing_te_f_num = housing_te_features.drop("ocean_proximity", axis=1)
X_test_prepared = imputer.transform(housing_te_f_num)
housing_te_fe = pd.DataFrame(
    X_test_prepared, columns=housing_te_f_num.columns, index=housing_te_features.index
)
housing_te_fe["rooms_per_household"] = (
    housing_te_fe["total_rooms"] / housing_te_fe["households"]
)
housing_te_fe["bedrooms_per_room"] = (
    housing_te_fe["total_bedrooms"] / housing_te_fe["total_rooms"]
)
housing_te_fe["population_per_household"] = (
    housing_te_fe["population"] / housing_te_fe["households"]
)

housing_te_cat = housing_te_features[["ocean_proximity"]]
housing_te_features = housing_te_fe.join(pd.get_dummies(housing_te_cat, drop_first=True))
housing_te_features['ocean_proximity_ISLAND'] = 0

In [ ]:

# train_X, train_y = (
#     sales_df_train
    
#     # split the dataset to train and test
#     .get_features_targets(target_column_names=target_col)
# )
save_dataset(context, housing_tr_features, 'train/housing/features')
save_dataset(context, housing_tr_target, 'train/housing/target')


# test_X, test_y = (
#     sales_df_test
    
#     # split the dataset to train and test
#     .get_features_targets(target_column_names=target_col)
# )
save_dataset(context, housing_te_features, 'test/housing/features')
save_dataset(context, housing_te_target, 'test/housing/target')

In [ ]:
housing_te_features